In [ ]:
from IPython.display import clear_output
!pip install numpy requests nlpaug
!pip install nltk
!pip install azure-ai-translation-text
!pip install arabert
!pip install tqdm
clear_output()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from arabert.preprocess import ArabertPreprocessor
import nlpaug.augmenter.word.context_word_embs as naw
from tqdm import tqdm
import numpy as np
from google.colab import drive
drive.mount('Drive')

Mounted at Drive


In [ ]:
whole_dataset=pd.read_csv('/content/Drive/MyDrive/SeniorProject1/data/Final_Full_Dataset.csv').drop(columns=['Unnamed: 0'])
whole_dataset

,ID,Tweet,Class
0,1444325619513401088,زيارتي للجناح السعودي إكسبو دبي أكبر جناح المع...,positive
1,1451962946461052928,زيارتي لجناح المملكة إكسبو يأخذك رحلة ماضي وحا...,positive
2,1487417431262117888,أثناء زيارتي جناح ألمانيا إكسبو دبي أكبر الأجن...,positive
3,1454025160659508992,زيارتي لجناح سيشل ولقائي رئيسها بجناحهم إكسبو ...,positive
4,1509228218733276928,جزء زيارتي للجناح الوطني الامارات فعلا منطقة ا...,positive
...,...,...,...
8347,1510700000000000000,اغلق معرض اكسبو الامارات اجلته الجاءحة ابوا...,neutral
8348,1510690000000000000,اختام معرض اكسبو دبي يومين الخميس فرصة المعرض...,positive
8349,1510690000000000000,هاشتاق عربي انتهاءه الامارات تكشف عد زوار اكسب...,neutral
8350,1510690000000000000,الحمدله انجاز راءع شكرا الامارات الحبيبه نبارك...,positive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
whole_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8352 entries, 0 to 8351
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      8352 non-null   int64 
 1   Tweet   8351 non-null   object
 2   Class   8352 non-null   object
dtypes: int64(1), object(2)
memory usage: 195.9+ KB


In [ ]:
whole_dataset.isnull().sum()

ID       0
Tweet    1
Class    0
dtype: int64

In [ ]:
whole_dataset.dropna( inplace=True)

In [ ]:
X=whole_dataset['Tweet'].values
y=whole_dataset['Class'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42,stratify=y)

In [ ]:
print(f'Total number of samples is {len(X)}:\n')
print('Number of Training data is ',len(X_train))
train_pos,train_neu,train_neg=sum(y_train=='positive'),sum(y_train=='neutral'),sum(y_train=='negative')
print(f'with positive class is {train_pos}, neutral class is {train_neu}, negative class is {train_neg}\n')
print('Number of Testing data is ',len(X_test))
test_pos,test_neu,test_neg=sum(y_test=='positive'),sum(y_test=='neutral'),sum(y_test=='negative')
print(f'with positive class is {test_pos}, neutral class is {test_neu}, negative class is {test_neg}')

Total number of samples is 8351:

Number of Training data is  7515
with positive class is 4252, neutral class is 2770, negative class is 493

Number of Testing data is  836
with positive class is 473, neutral class is 308, negative class is 55


In [ ]:
training_set=pd.DataFrame({'Tweet':X_train,'Class':y_train})
training_set

,Tweet,Class
0,لـا توجد گلمات تعبر مشاعر گل اماراتي اثبت اكسب...,positive
1,تراس سعادة المهندس ابراهيم رمل الرءيس التنفيذي...,positive
2,اختم فريق فرسان الامارات لاستعراضات الجوية الت...,positive
3,اكسبو دبي يستضيف قادة السياسة والاعمال والثقاف...,neutral
4,شكرا لكل خط ونفذ وساعد علا انجاح اكسبو دبي ال...,positive
...,...,...
7510,جولة الاتحاد بالجناح الاوكراني اكسبو دبي نتصدر...,neutral
7511,ليلة لتاريخ يكرم ويشكر المنتسبين المشروع الوطن...,positive
7512,مقر إكسبو دبي كرمنا الفائزين بجائزة محمد راشد ...,positive
7513,التدافع اللي صاير إكسبو حفلة محزن بنفس مب مستغ...,negative


In [ ]:
testing_set=pd.DataFrame({'Tweet':X_test,'Class':y_test})
testing_set

,Tweet,Class
0,بالغ الاسف باقي يومين علا الي ماراح مازالت ال...,positive
1,وعدنا العالم بانا سنبهره ونءكد وعدنا ونعد ب...,positive
2,مصر تطرح مشروعات عملاقة بقطاع الزراعة مشاركتها...,neutral
3,متطوعون حب اكسبو وداعنا اكسبو وقضيت اجمل شهور ...,positive
4,يسوا ميحد,positive
...,...,...
831,دوزي ينشر فيديو حفله الاخير اكسبو,neutral
832,استمتعت وايد اني بكون حزين لان بيسكرون استمت...,positive
833,فخورة حقته بلادي اكسبو دبي المدة القصيرة اشه...,positive
834,دبي انفجارات اكسبو عنها ناشطون ساعات نتيجة عمل...,negative


In [ ]:
#training_set.to_csv('experiment/training_set.csv')
#testing_set.to_csv('experiment/testing_set.csv')

### Data Augmentation
on the negative samples

In [ ]:
negative_data=training_set[training_set['Class']=='negative']['Tweet']
negative_data

9       عاجل عطية ينبش جناح الأردن اكسبو دبي ويؤكد فشل...
17       لاسف السوق حق فشل ذريع اكسبو غيره بسب فقدان ث...
22                         اكسبو دبي وربي ياهو زحمه طبيعي
26      طلاب المدارس الامارات  اكسبو  يردون اهزوجة جما...
30                                    دكان قراشي مب اكسبو
                              ...                        
7446    نءيد ظرب ارامكوالنفطية محطةتحليةالمياة محطةتول...
7456                             لحد يجي اكسبو لان حر موت
7466                        دبي ارتفاع الاسعار بسب اكسبو 
7504                            شو زحمة اكسبو    شو      
7513    التدافع اللي صاير إكسبو حفلة محزن بنفس مب مستغ...
Name: Tweet, Length: 493, dtype: object

##### Generate New Negative Examples by AraBERT

In [ ]:
aug = naw.ContextualWordEmbsAug(
    model_path='aubmindlab/bert-base-arabert', action="substitute")
preprocessor=ArabertPreprocessor(model_name='aubmindlab/bert-base-arabert')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/717k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


100%|██████████| 241M/241M [04:10<00:00, 964kiB/s] 


[2024-04-20 10:17:42,149 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [ ]:
generated_data=[]
for text in tqdm(negative_data):
  generated_text = aug.augment(text,n=2)
  generated_text=list(map(lambda strText: preprocessor.unpreprocess(strText),generated_text))
  generated_data.extend(generated_text)

generated_data[:3]

100%|██████████| 493/493 [41:08<00:00,  5.01s/it]


['إذ هل ينبش جبل الأردن عله يعلن فشل نجاح الوقائع والاخبارية؟ و',
 'لعطية لماذا جناح ساكسبو؟ ويؤكدان ومهازل تحديات التي عن ل',
 'مع كحق فشل ذريع الغيره بسب كل سوق المتداولين سوق [رابط] دبي [UNK]']

In [ ]:
aug_neg_data=pd.concat([negative_data,pd.Series(generated_data)])
aug_neg_data.to_csv('aug_neg_data.csv')
aug_neg_data

9      عاجل عطية ينبش جناح الأردن اكسبو دبي ويؤكد فشل...
17      لاسف السوق حق فشل ذريع اكسبو غيره بسب فقدان ث...
22                        اكسبو دبي وربي ياهو زحمه طبيعي
26     طلاب المدارس الامارات  اكسبو  يردون اهزوجة جما...
30                                   دكان قراشي مب اكسبو
                             ...                        
981                                      دبي فببسب [UNK]
982                                          شوان اكسبو»
983                                        شو زحمة عام «
984    م اللي صاير إكسبوها محزن ذلك وببدون من واليومي...
985    التدافعة صاير بين شأن محزن و مب مستغرب بدوننا ...
Length: 1479, dtype: object

In [ ]:
aug_neg_data=pd.read_csv('aug_neg_data.csv').drop(columns=['Unnamed: 0']).values
aug_neg_data

array([['عاجل عطية ينبش جناح الأردن اكسبو دبي ويؤكد فشل ومهازل الوقائع الوقائع الاخبارية الاردن الوقائع'],
       [' لاسف السوق حق فشل ذريع اكسبو غيره بسب فقدان ثقة المتداولين سوق دبي تبخر اموالهم'],
       ['اكسبو دبي وربي ياهو زحمه طبيعي'],
       ...,
       ['شو زحمة عام «'],
       ['م اللي صاير إكسبوها محزن ذلك وببدون من واليومية مثلما نشوفها بثقافة إحترام الطابور في خصوصاها'],
       ['التدافعة صاير بين شأن محزن و مب مستغرب بدوننا عن اليومية» تبقى بثقافة إحترام لن تنتهي خصوصا أن']],
      dtype=object)

###### Back-translation

In [ ]:
from azure.ai.translation.text import TextTranslationClient, TranslatorCredential
from azure.ai.translation.text.models import InputTextItem
from azure.core.exceptions import HttpResponseError

In [ ]:
source_lang = 'ar'  # Original language is Arabic
target_lang = 'en'  # Translate to English
key='942c029112ab4fe0924ca25a31d2018d'
regoin= "qatarcentral"
text_translator = TextTranslationClient(credential = TranslatorCredential(key, regoin));


def translate_text(text, source_lang, target_lang):
    try:
        source_language = source_lang
        target_languages = [target_lang]
        input_text_elements = [ InputTextItem(text = text[0]) ]

        response = text_translator.translate(content = input_text_elements, to = target_languages, from_parameter = source_language)
        translation = response[0] if response else None
        translated_texts=[]
        if translation:
            for translated_text in translation.translations:
              translated_texts.append(translated_text.text)

    except HttpResponseError as exception:
        print(f"Error Code: {exception.error.code}")
        print(f"Message: {exception.error.message}")
    return translated_texts

In [ ]:
translated_data=[]
for text in aug_neg_data[1478]:
  english_translation = translate_text(text, source_lang, target_lang)  # Translate from Arabic to English
  back_translation = translate_text(english_translation, target_lang, source_lang)  # Translate back to Arabic
  translated_data.extend(back_translation)

translated_data[:3]

['أنا أبو سليمان أنت تقول هذا عزيزا عن أولويتنا في',
 'وجود تزوير كبير وتشويه متعمد ولكن انا اكسبو غير انها ليست',
 'مذيع صوت [مستعمل] والمقصود [UNK] صورة [UNK] شبكة إكسبو دبي؟']

In [ ]:
translated_data=pd.Series(translated_data)
translated_data

0     عطية عاجلة تنبش جناح معرض إكسبو الأردن دبي وتؤ...
1      للأسف السوق فشل كارثي اكسبو الاخرين بسبب فقدا...
2                    إكسبو دبي أقسم بالله أنه حشد طبيعي
3     طلاب مدارس الإمارات إكسبو يريدون الزواج من مشج...
4                                  متجر كراشي ليس إكسبو
                            ...                        
46                                       دبي VBSP [UNK]
47                                          شوان إكسبو»
48                             يا لها من حركة مرور عامة
49    ما حدث هو أنهم اكتسبوها من المحزن أنه وبدون من...
50    التدافع بين أمر محزن وليس مستغربا بدوننا من ال...
Length: 1478, dtype: object

In [ ]:
translated_data.to_csv('translated_data.csv')

In [ ]:
aug_neg_data=pd.DataFrame({'Tweet':np.concatenate([aug_neg_data.reshape(-1,),translated_data.values])})
aug_neg_data

,Tweet
0,عاجل عطية ينبش جناح الأردن اكسبو دبي ويؤكد فشل...
1,لاسف السوق حق فشل ذريع اكسبو غيره بسب فقدان ث...
2,اكسبو دبي وربي ياهو زحمه طبيعي
3,طلاب المدارس الامارات اكسبو يردون اهزوجة جما...
4,دكان قراشي مب اكسبو
...,...
2952,دبي VBSP [UNK]
2953,شوان إكسبو»
2954,يا لها من حركة مرور عامة
2955,ما حدث هو أنهم اكتسبوها من المحزن أنه وبدون من...


In [ ]:
aug_neg_data.to_csv('aug_neg_samples.csv')

###### preprocess augemented samples

In [ ]:
from IPython.display import clear_output
!pip install pyarabic
!pip install emoji
clear_output()
import re
import pandas as pd
import matplotlib.pyplot as plt
from pyarabic import araby
from nltk.corpus import stopwords
import nltk
import emoji
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
arabic_emoji = pd.read_csv('/content/Drive/MyDrive/SeniorProject1/data/arabic_emojis.csv')
UNICODE_EMOJI = dict(map(lambda i,j : (i,j) , list(arabic_emoji['emoji']),list(arabic_emoji['text'])))

In [ ]:
#stopwords
ar_stops = set(stopwords.words('arabic'))
stop_words = {"،","","ورحمه","وبركاته","عليكم","السلام","آض","آمينَ","آه","آهاً","آي","أ","أب","أجل","أجمع","أخ","أخذ","أصبح","أضحى","أقبل",
"أقل","أكثر","ألا","أم","أما","أمامك","أمامكَ","أمسى","أمّا","أن","أنا","أنت","أنتم","أنتما","أنتن","أنتِ","أنشأ","أنّى","أو","أوشك","أولئك",
"أولئكم","أولاء","أولالك","أوّهْ","أي","أيا","أين","أينما","أيّ","أَنَّ","أََيُّ","أُفٍّ","إذ","إذا","إذاً","إذما","إذن","إلى","إليكم","إليكما","إليكنّ",
"إليكَ","إلَيْكَ","إلّا","إمّا","إن","إنّما","إي","إياك","إياكم","إياكما","إياكن","إيانا","إياه","إياها","إياهم","إياهما","إياهن","إياي","إيهٍ","إِنَّ","ا","ابتدأ",
"اثر","اجل","احد","اخرى","اخلولق","اذا","اربعة","ارتدّ","استحال","اطار","اعادة","اعلنت","اف","اكثر","اكد","الألاء","الألى","الا","الاخيرة","الان",
"الاول","الاولى","التى","التي","الثاني","الثانية","الذاتي","الذى","الذي","الذين","السابق","الف","اللائي","اللاتي","اللتان","اللتيا","اللتين",
"اللذان","اللذين","اللواتي","الماضي","المقبل","الوقت","الى","اليوم","اما","امام","امس","ان","انبرى","انقلب","انه","انها","او","اول","اي",
"ايار","ايام","ايضا","ب","بات","باسم","بان","بخٍ","برس","بسبب","بسّ","بشكل","بضع","بطآن","بعد","بعض","بك","بكم","بكما","بكن","بل","بلى","بما",
"بماذا","بمن","بن","بنا","به","بها","بي","بيد","بين","بَسْ","بَلْهَ","بِئْسَ","تانِ","تانِك","تبدّل","تجاه","تحوّل","تلقاء","تلك","تلكم","تلكما","تم",
"تينك","تَيْنِ","تِه","تِي","ثلاثة","ثم","ثمّ","ثمّة","ثُمَّ","جعل","جلل","جميع","جير","حار","حاشا","حاليا","حاي","حتى","حرى","حسب","حم","حوالى","حول",
"حيث","حيثما","حين","حيَّ","حَبَّذَا","حَتَّى","حَذارِ","خلا","خلال","دون","دونك","ذا","ذات","ذاك","ذانك","ذانِ","ذلك","ذلكم","ذلكما","ذلكن","ذو","ذوا",
"ذواتا","ذواتي","ذيت","ذينك","ذَيْنِ","ذِه","ذِي","راح","رجع","رويدك","ريث","رُبَّ","زيارة","سبحان","سرعان","سنة","سنوات","سوف","سوى","سَاءَ","سَاءَمَا",
"شبه","شخصا","شرع","شَتَّانَ","صار","صباح","صفر","صهٍ","صهْ","ضمن","طاق","طالما","طفق","طَق","ظلّ","عاد","عام","عاما","عامة","عدا","عدة","عدد","عدم",
"عسى","عشر","عشرة","علق","على","عليك","عليه","عليها","علًّ","عن","عند","عندما","عوض","عين","عَدَسْ","عَمَّا","غدا","غير","ـ","ف","فان","فلان","فو",
"فى","في","فيم","فيما","فيه","فيها","قال","قام","قبل","قد","قطّ","قلما","قوة","كأنّما","كأين","كأيّ","كأيّن","كاد","كان","كانت","كذا","كذلك","كرب",
"كل","كلا","كلاهما","كلتا","كلم","كليكما","كليهما","كلّما","كلَّا","كم","كما","كي","كيت","كيف","كيفما","كَأَنَّ","كِخ","لئن","لا","لات","لاسيما","لدن","لدى",
"لعمر","لقاء","لك","لكم","لكما","لكن","لكنَّما","لكي","لكيلا","للامم","لم","لما","لمّا","لن","لنا","له","لها","لو","لوكالة","لولا","لوما","لي","لَسْتَ",
"لَسْتُ","لَسْتُم","لَسْتُمَا","لَسْتُنَّ","لَسْتِ","لَسْنَ","لَعَلَّ","لَكِنَّ","لَيْتَ","لَيْسَ","لَيْسَا","لَيْسَتَا","لَيْسَتْ","لَيْسُوا","لَِسْنَا","ما","ماانفك","مابرح","مادام","ماذا",
"مازال","مافتئ","مايو","متى","مثل","مذ","مساء","معاذ","مقابل","مكانكم","مكانكما","مكانكنّ","مكانَك","مليار","مليون","مما","ممن","من","منذ",
"منها","مه","مهما","مَنْ","مِن","نحن","نحو","نعم","نفس","نفسه","نهاية","نَخْ","نِعِمّا","نِعْمَ","ها","هاؤم","هاكَ","هاهنا","هبّ","هذا","هذه","هكذا",
"هل","هلمَّ","هلّا","هم","هما","هن","هنا","هناك","هنالك","هو","هي","هيا","هيت","هيّا","هَؤلاء","هَاتانِ","هَاتَيْنِ","هَاتِه","هَاتِي","هَجْ","هَذا","هَذانِ","هَذَيْنِ",
"هَذِه","هَذِي","هَيْهَاتَ","و","و6","وا","واحد","واضاف","واضافت","واكد","وان","واهاً","واوضح","وراءَك","وفي","وقال","وقالت","وقد","وقف","وكان","وكانت",
"ولا","ولم","ومن","مَن","وهو","وهي","ويكأنّ","وَيْ","وُشْكَانََ","يكون","يمكن","يوم","ّأيّان"}

def clean_data(text):
  text=str(text)
  #remove repeated words of  ا و ي ح خ ر
  text=re.sub(r'([اويخحر])\1+', r'\1', text)
  #remove urls
  text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', text)
  #remove punctuations
  text = re.sub('[_]', ' ',  text)
  text = re.sub(r'[!()-{};:\,<>./?@#$%^&*_~\n\t""،]',' ',text)
  #remove non-arabic characters & digits
  text =re.sub(r'[^0-9\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD.0-9٠-٩]+', ' ',text)
  #remove @usernames
  text = re.sub('@[^\s]+', ' ', text)
  #remove laughs of any long and normalized it to ههه
  text= re.sub('[ه]{3,}', 'ههه', text)
  #remove diacritics
  text = araby.strip_diacritics(text)
  #remove tashkeel
  text = araby.strip_tashkeel(text)
  #remove tatweel
  text = araby.strip_tatweel(text)
  #remove stop
  tokens=word_tokenize(text)
  text=" ".join([w.strip() for w in tokens if not w in ar_stops and not w in stop_words and len(w) >= 2])

  #convert emoji to text
  for emot in UNICODE_EMOJI:
      text = ''.join("_".join(UNICODE_EMOJI[emot].replace(",", "").replace(":", "").split()) + " " if emot in UNICODE_EMOJI else emot for emot in text)
  #remove other emojis
  text = emoji.replace_emoji(text, replace='')

  return text


In [ ]:
aug_neg_data['Tweet']=aug_neg_data['Tweet'].apply(lambda text: clean_data(text))
aug_neg_data

,Tweet
0,عاجل عطية ينبش جناح الأردن اكسبو دبي ويؤكد فشل...
1,لاسف السوق حق فشل ذريع اكسبو غيره بسب فقدان ثق...
2,اكسبو دبي وربي ياهو زحمه طبيعي
3,طلاب المدارس الامارات اكسبو يردون اهزوجة جماهي...
4,دكان قراشي مب اكسبو
...,...
2952,دبي
2953,شوان إكسبو
2954,حركة مرور
2955,حدث أنهم اكتسبوها المحزن أنه وبدون ويوميا نراه...


In [ ]:
aug_neg_data['Class']='negative'
aug_neg_data

,Tweet,Class
0,عاجل عطية ينبش جناح الأردن اكسبو دبي ويؤكد فشل...,negative
1,لاسف السوق حق فشل ذريع اكسبو غيره بسب فقدان ثق...,negative
2,اكسبو دبي وربي ياهو زحمه طبيعي,negative
3,طلاب المدارس الامارات اكسبو يردون اهزوجة جماهي...,negative
4,دكان قراشي مب اكسبو,negative
...,...,...
2952,دبي,negative
2953,شوان إكسبو,negative
2954,حركة مرور,negative
2955,حدث أنهم اكتسبوها المحزن أنه وبدون ويوميا نراه...,negative


In [ ]:
training_set.drop(training_set.index[training_set['Class']=='negative'],inplace=True)

In [ ]:
training_set=pd.concat([training_set,aug_neg_data])
training_set

,Tweet,Class
0,شكرا سيدي سمو الشيخ نهيان مبارك ال نهـ...,positive
1,اكسبو دبي فخامة الحدث وابداع التنظيم ونجاح منق...,positive
2,اختم فريق فرسان الامارات لاستعراضات الجوية الت...,positive
3,رحلة مبهرة لتنقل البشرية جناح التنقل بـ اكسبو ...,neutral
4,وداعا اكسبو,positive
...,...,...
2952,دبي,negative
2953,شوان إكسبو,negative
2954,حركة مرور,negative
2955,حدث أنهم اكتسبوها المحزن أنه وبدون ويوميا نراه...,negative


##### Downsample the Positive Samples

In [ ]:
from sklearn.utils import resample

In [ ]:
training_set.to_csv('tem_training.csv')

In [ ]:
training_set=pd.read_csv('tem_training.csv').drop(columns=['Unnamed: 0'])
training_set

,Tweet,Class
0,شكرا سيدي سمو الشيخ نهيان مبارك ال نهـ...,positive
1,اكسبو دبي فخامة الحدث وابداع التنظيم ونجاح منق...,positive
2,اختم فريق فرسان الامارات لاستعراضات الجوية الت...,positive
3,رحلة مبهرة لتنقل البشرية جناح التنقل بـ اكسبو ...,neutral
4,وداعا اكسبو,positive
...,...,...
9974,دبي,negative
9975,شوان إكسبو,negative
9976,حركة مرور,negative
9977,حدث أنهم اكتسبوها المحزن أنه وبدون ويوميا نراه...,negative


In [ ]:
training_set['Class'].value_counts()

Class
positive    4253
negative    2957
neutral     2769
Name: count, dtype: int64

In [ ]:
pos_samples=training_set.iloc[training_set.index[training_set['Class']=='positive']]
len(pos_samples)

4253

In [ ]:
pos_samples

,Tweet,Class
0,شكرا سيدي سمو الشيخ نهيان مبارك ال نهـ...,positive
1,اكسبو دبي فخامة الحدث وابداع التنظيم ونجاح منق...,positive
2,اختم فريق فرسان الامارات لاستعراضات الجوية الت...,positive
4,وداعا اكسبو,positive
6,المستشار الديني بديوان ولي عهد ابوظبي وامين جا...,positive
...,...,...
7014,اشارك معكم صوري وانا اخر اكسبو دبي تمنيت بمقدو...,positive
7015,استمتع زوار إكسبو دبي بعرض رائع للفقاعات العمل...,positive
7019,حاكم دبي يشهد اطلاق النسخة تحدي تكنولوجيا الغذ...,positive
7020,وداعا اكسبو قصة عظيمة اثرها باقي يزال وحياكم ا...,positive


In [ ]:
resampled_pos=resample(pos_samples,replace=False,n_samples=sum(training_set['Class']=='negative'),random_state=42)
resampled_pos

,Tweet,Class
308,السناب مولع ممتع هالفترة لانه اغلب دول الخليج ...,positive
4239,الاحتفالات الداءمة والمرح الغير محدود حفلات وج...,positive
4185,دبي دوما تصنع المعجزات لاسف استطع الذهاب ورءية...,positive
4439,يواصل اكسبو انجازاته ونشاطاته الفنية باستقبال...,positive
1547,اجواء طربية غلفت مسرح اكسبو بليله يغمرها الدفء...,positive
...,...,...
6530,فعلا والله افكر بيني نفسي بنفس هالفكرة تخيلوا ...,positive
1507,تنظيم إمارة دبي اكسبو دبي يفوق الخيال جائحة كو...,positive
4417,الیة مریحة لطي الانزلاق تلاءم الظروف الجویة نو...,positive
3943,زحمة وطابور طويل محل البيك اكسبو دبي رافع راسن...,positive


In [ ]:
training_set.drop(training_set.index[training_set['Class']=='positive'],inplace=True)
training_set=pd.concat([training_set,resampled_pos])
training_set

,Tweet,Class
3,رحلة مبهرة لتنقل البشرية جناح التنقل بـ اكسبو ...,neutral
5,خصوصیة اضافیة بناءھا منفصلة مكان اقامتك الرءیس...,neutral
7,احضان غابة مقلوبة وسيمفونية ملحمية ينتظرك جناح...,neutral
8,مدير مكتب برنامج الاغذية العالمي التابع المتحد...,neutral
14,فاز جناح المكسيك بالمركز التصميم اكسبو دبي,neutral
...,...,...
6530,فعلا والله افكر بيني نفسي بنفس هالفكرة تخيلوا ...,positive
1507,تنظيم إمارة دبي اكسبو دبي يفوق الخيال جائحة كو...,positive
4417,الیة مریحة لطي الانزلاق تلاءم الظروف الجویة نو...,positive
3943,زحمة وطابور طويل محل البيك اكسبو دبي رافع راسن...,positive


In [ ]:
training_set['Class'].value_counts()

Class
negative    2957
positive    2957
neutral     2769
Name: count, dtype: int64

In [ ]:
print(f'The total number of the training set recoreds after downsampling is {len(training_set)}')

The total number of the training set recoreds after downsampling is 8683


##### save the training and testing sets

In [ ]:
training_set.to_csv('/content/Drive/MyDrive/SeniorProject1/data/experiment/training_set.csv')
testing_set.to_csv('/content/Drive/MyDrive/SeniorProject1/data/experiment/testing_set.csv')